In [4]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [5]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import xgboost as xgb
from tqdm import tqdm
from nltk import word_tokenize
import nltk

In [6]:
from sklearn.model_selection import train_test_split
from nltk import word_tokenize

In [7]:
test=pd.read_csv('test_text.csv',sep="\|\|", names=["ID","Text"], skiprows=1,header=None)
train=pd.read_csv('training_text.csv',sep="\|\|", names=["ID","Text"],skiprows=1,header=None)

C:\Users\root\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
C:\Users\root\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [8]:
test_variants=pd.read_csv('test_variants.csv',sep=",")
training_variants=pd.read_csv('training_variants.csv',sep=",")
# frames=[training_text,training_variants]
# frames1=[test_text,test_variants]

In [9]:
# train = pd.concat(frames,axis=1,join='outer')
# test=pd.concat(frames1,axis=1,join='outer')

In [10]:
# #label encoding if needed.
# lbl_enc = preprocessing.LabelEncoder()
# y = lbl_enc.fit_transform(training_text.Class.values) 
#string is any column with string values

In [11]:
train['Class'] = training_variants['Class']

In [12]:
y = train.Class

In [13]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.Text.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

## Building Basic Models

#  TF-IDF (Term Frequency - Inverse Document Frequency) followed by a simple Logistic Regression

In [14]:
# Always start with these features. They work (almost) everytime!
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

In [15]:
xtrain = ['-99999' if x is np.nan else x for x in xtrain]
xvalid = ['-99999' if x is np.nan else x for x in xvalid]

In [16]:
tfv.fit(list(xtrain) + list(xvalid))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=3,
        ngram_range=(1, 3), norm='l2', preprocessor=None, smooth_idf=1,
        stop_words='english', strip_accents='unicode', sublinear_tf=1,
        token_pattern='\\w{1,}', tokenizer=None, use_idf=1,
        vocabulary=None)

In [17]:
xtrain_tfv = tfv.transform(xtrain)
xvalid_tfv = tfv.transform(xvalid)

In [19]:
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [20]:
predictions = clf.predict_proba(xvalid_tfv)

In [22]:
#imported from kaggle. It is just evaluation metric
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        print(actual2.shape)
        actual=actual.reshape(actual2.shape[0],1)
        print(actual.shape)
        for i, val in enumerate(actual2.astype(int)):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [23]:
print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

(333, 9)
(333, 1)
logloss: 2.013 
